## REFL Assignment 1
Vinodh Chincholi, MDS202252

Question 1(ii) Value Search for 687 gridworld problem

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
state_set = [(i,j) for i in range(5) for j in range(5) 
                  if not((i==2 and j==2) or (i==3 and j==2))]

action_map = {'up':(-1,0), 'down':(1,0), 'left':(0,-1), 'right':(0,1), 'stay':(0,0)}

gamma = 0.9

values = np.random.normal(0, 1, (5,5))
action = np.zeros((5,5))

In [4]:
def reward(state):
    if state == (4,2):
        return -10
    elif state == (4,4):
        return 10
    else:
        return 0
    
def is_legal(state, action):
    candState = tuple(map(sum, zip(state, action)))
    if candState in state_set:
        return True
    else:
        return False
    
def summation(state, action):
    
    output = 0
    
    if state == (4,4):
        return output
    
    if action in ['up', 'down']:
        weight = {action:0.8, 'left':0.05, 'right':0.05, 'stay':0.1}
    elif action in ['left', 'right']:
        weight = {action:0.8, 'up':0.05, 'down':0.05, 'stay':0.1}
        
    for a in weight:
        a_tup = action_map[a]
        candState = tuple(map(sum, zip(state, a_tup)))
        if candState in state_set:
            output += weight[a]*(reward(candState) + gamma*values[candState])
            
    return output

In [5]:
## Using equation from class to find policy at each state

for _ in range(1000):
    for state in state_set:
        all_actions = [summation(state, action) for action in ['up', 'down', 'left', 'right']]
        action[state] = np.argmax(all_actions)
        values[state] = max(all_actions)

In [14]:
action2 = [['_' for i in range(5)] for j in range(5)]
actions = ['u', 'd', 'l', 'r']

for state in state_set:
    i,j = state[0],state[1]
    action2[i][j] = actions[int(action[i,j])]
    
action2[4][4] = '_'

In [15]:
## Final policy at each state

action2

[['r', 'd', 'r', 'd', 'd'],
 ['r', 'r', 'r', 'd', 'd'],
 ['r', 'u', '_', 'd', 'd'],
 ['u', 'u', '_', 'r', 'd'],
 ['u', 'u', 'r', 'r', '_']]